In [2]:
# noexport

!typech evaluation_utils.ipynb


[NbConvertApp] Converting notebook evaluation_utils.ipynb to python


In [17]:
from train_utils import *

In [16]:
from getsecret import getsecret
import subprocess
import json

In [27]:
#print(get_models_currently_training())

In [15]:
def get_models_currently_training():
  output = []
  local_info = json.load(open('current.json'))
  output.append(local_info['base_path'])
  remote_info = json.loads(subprocess.check_output(getsecret('REMOTE_SSH_COMMAND'), shell=True))
  output.append(remote_info['base_path'])
  return output

In [9]:
from glob import glob
import os
from os import path

In [11]:
def find_last_completed_epoch(model_name):
  return len(glob(path.join(model_name, 'info_*.json')))


In [22]:


def get_models_with_training_status():
  output = []
  models_currently_training = set(get_models_currently_training())
  for x in glob('tm_*'):
    model_status = {
      'name': x,
      'epochs_done': find_last_completed_epoch(x),
      'status': 'incomplete'
    }
    status_file_path = path.join(x, 'status.json')
    if not os.path.exists(status_file_path):
      output.append(model_status)
      continue
    status_info = json.load(open(status_file_path))
    if status_info['status'] == 'done':
      #model_status['epochs_done'] = find_last_completed_epoch(x)
      model_status['status'] = status_info['status']
      output.append(model_status)
      continue
    if status_info['status'] == 'training':
      if x in models_currently_training:
        model_status['status'] = status_info['status']
      output.append(model_status)
      continue
    raise Exception('invalid status for model', x, status_info['status'])
    #output.append(model_status)
  return output

In [26]:
import shutil

def remove_incomplete_models():
  for status_info in get_models_with_training_status():
    if status_info['epochs_done'] == 0 and status_info['status'] == 'incomplete':
      model_name = status_info['name']
      shutil.rmtree(model_name)

remove_incomplete_models()

In [22]:
def get_parameter_info_list_for_model(model_name):
  parameters_file_path = path.join(x, 'parameters.json')
  return json.load(open(parameters_file_path))

def get_parameter_map_for_model(model_name):
  output = {}
  for parameter_info in get_parameter_info_list_for_model(model_name):
    parameter_name = parameter_info['name']
    parameter_value = parameter_info['value']
    output[parameter_name] = parameter_value
  return output

def list_models_finished_training():
  output = []
  for x in glob('tm_*'):
    status_file_path = path.join(x, 'status.json')
    if not os.path.exists(status_file_path):
      continue
    status_info = json.load(open(status_file_path))
    #print(status_info)
    if status_info['status'] != 'done':
      continue
    output.append(x)
  return output

def list_models_matching_parameters(parameters_map):
  for model_name in list_models_finished_training():
    parameters_map = get_parameter_map_for_model(model_name)

for x in list_models_finished_training():
  get_parameter_map_for_model(x))

{'dataset_name': '2019_04_01', 'model_name': 'selfattentionlstm', 'criterion': 'NLLLoss', 'learning_rate': 0.0005, 'window_embed_size': 256, 'difficulty': True, 'time_of_day': True, 'day_of_week': True, 'domain_productivity': True, 'domain_category': True, 'initial_difficulty': True, 'languages': True, 'num_prior_entries': 10, 'sample_every_n_visits': 1, 'sample_difficulty_every_n_visits': 1, 'disable_prior_visit_history': True, 'disable_difficulty_history': True, 'enable_current_difficulty': True, 'num_features': 277}
{'dataset_name': '2019_04_01', 'model_name': 'selfattentionlstm', 'criterion': 'NLLLoss', 'learning_rate': 0.005, 'window_embed_size': 512, 'difficulty': True, 'time_of_day': True, 'day_of_week': True, 'domain_productivity': True, 'domain_category': True, 'initial_difficulty': True, 'languages': True, 'num_prior_entries': 30, 'sample_every_n_visits': 1, 'sample_difficulty_every_n_visits': 1, 'disable_prior_visit_history': True, 'disable_difficulty_history': True, 'enable